In [0]:
import os

#Google Colab -DB 연결을 위한 파이썬 라이브러리 설치
!pip3 install sqlalchemy pymysql

#Konlpy 설치 (feat.JPype1-py3)
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip3 install JPype1-py3
!pip3 install konlpy
!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

#폴더 만들기
!mkdir mecab 

#mecab-ko 설치
os.chdir('/content/mecab')
!wget "https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz"
!tar zxfv mecab-0.996-ko-0.9.2.tar.gz
!rm -rf mecab-0.996-ko-0.9.2.tar.gz
os.chdir('./mecab-0.996-ko-0.9.2')
!./configure
!make
!make check
!make install

#mecab-ko-dic 설치
os.chdir('/content/mecab')
!wget "https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.0.3-20170922.tar.gz"
!tar zxfv mecab-ko-dic-2.0.3-20170922.tar.gz
!rm -rf mecab-ko-dic-2.0.3-20170922.tar.gz
os.chdir('./mecab-ko-dic-2.0.3-20170922')
!apt-get install automake1.11
!./autogen.sh
!./configure
!ldconfig
!make
!make install

#mecab-python3 설치
os.chdir('/content/mecab')
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

In [0]:
!pip install pymysql pytagcloud pygame simplejson
!pwd
!wget http://cdn.naver.com/naver/NanumFont/fontfiles/NanumFont_TTF_ALL.zip
!unzip NanumFont_TTF_ALL.zip -d -f .
!rm -f NanumFont_TTF_ALL.zip
!find / -name "pytagcloud"
!ls /usr/local/lib/python3.6/dist-packages/pytagcloud/fonts
!cp /content/mecab/NanumGothic.ttf /usr/local/lib/python3.6/dist-packages/pytagcloud/fonts

In [0]:
#pymysql로 MySQL openlawdb 데이터베이스에 연결하기
import pymysql
con = pymysql.connect(
        host='<your-db-endpoint>',
        port=3306,
        user='<your-db-user>',
        passwd='<your-db-password>',
        db='<your-db-databasename>',
        charset='utf8',
        )

#연결된 openlawdb에서 쿼리 날릴 준비
cs = con.cursor(pymysql.cursors.DictCursor)
#cs = con.cursor()

#실행하고 싶은 쿼리 세팅
cs.execute("select sentencedate, casecontent from casetbl where sentencedate between '2017-01-01' and '2017-02-01';")
#query = "select casecontent from casetbl where sentencedate between %s and %s;"
#cs.execute(query,('2017-01-01','2017-02-01'))

#세팅된 쿼리의 모든 결과값을 query_contentsOfcases(Dictionary)로 저장
query_contentsOfcases = cs.fetchall()

#MySQL openlawdb 연결종료
con.close()
print(query_contentsOfcases)

In [0]:
result_set_flatten = ''
for singlecase in query_contentsOfcases:
    result_set_flatten = result_set_flatten + singlecase['casecontent']
print(len(result_set_flatten))

In [0]:
from konlpy.tag import Mecab
from collections import Counter
import random
import pytagcloud # requires Korean font support
from konlpy.utils import pprint

r = lambda: random.randint(0,255)
color = lambda: (r(), r(), r())

def get_Mtags(text, ntags=50, multiplier=0.5):
    m = Mecab()
    nouns = m.nouns(text)
    count = Counter(nouns)
    return([{ 'color': color(), 'tag': n, 'size': int(c*multiplier) }\
                for n, c in count.most_common(ntags)])

def draw_cloud(tags, filename):
    pytagcloud.create_tag_image(tags, filename, size=(1024, 768), fontname='/content/mecab/NanumGothic')

In [0]:
Mtags = get_Mtags(result_set_flatten)
print(pprint(Mtags))
draw_cloud(Mtags, 'Mwordcloud.png')

In [0]:
from IPython.display import Image, display
display(Image(filename='Mwordcloud.png'))